<h1>Fake Python Web Scraping Bonus</h1>

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

<h3>1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.</h3>

In [2]:
url = 'https://www.billboard.com/charts/hot-100/'
response = requests.get(url)
soup = bs(response.text)

In [17]:
this_week_rows = soup.findAll('ul', {'class': 'o-chart-results-list-row'})
this_week_positions = [item.findChild('li').findChild('span').text.strip() for item in this_week_rows]

# ARTISTS
artist_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(1) > span')
artists = [artist.text.strip() for artist in artist_rows]

# SONGS
song_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(1) > h3')
songs = [song.text.strip() for song in song_rows]

# LAST WEEK
last_week_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(4) > span')
last_week_positions = [last_week.text.strip() for last_week in last_week_rows]

# PEAK POSITION
peak_position_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(5) > span')
peak_positions = [peak_position.text.strip() for peak_position in peak_position_rows]

# WEEKS ON CHART
weeks_on_chart_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(6) > span')
weeks_on_chart = [weeks_on_chart.text.strip() for weeks_on_chart in weeks_on_chart_rows]

# Create a dataframe with the extracted data
billboard_df = pd.DataFrame({
    'this_week': this_week_positions,
    'artist': artists,
    'song': songs,
    'last_week': last_week_positions,
    'peak_position': peak_positions,
    'weeks_on_chart': weeks_on_chart
})

print(billboard_df.head())


  this_week                       artist             song last_week  \
0         1                Morgan Wallen       Last Night         1   
1         2                          SZA        Kill Bill         2   
2         3                  Miley Cyrus          Flowers         3   
3         4  Eslabon Armado X Peso Pluma  Ella Baila Sola         4   
4         5          Rema & Selena Gomez        Calm Down         6   

  peak_position weeks_on_chart  
0             1             14  
1             1             21  
2             1             16  
3             4              7  
4             5             35  


<h3>2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.</h3>

In [19]:
def get_billboard_chart_data(url):
    # Get the response from the URL
    response = requests.get(url)
    soup = bs(response.text)

    # Get the list of rows
    this_week_rows = soup.findAll('ul', {'class': 'o-chart-results-list-row'})

    # Get the list of positions
    this_week_positions = [item.findChild('li').findChild('span').text.strip() for item in this_week_rows]

    # Get the list of artists
    artist_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(1) > span')
    artists = [artist.text.strip() for artist in artist_rows]

    # Get the list of songs
    song_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(1) > h3')
    songs = [song.text.strip() for song in song_rows]

    # Get the list of last week positions
    last_week_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(4) > span')
    last_week_positions = [last_week.text.strip() for last_week in last_week_rows]

    # Get the list of peak positions
    peak_position_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(5) > span')
    peak_positions = [peak_position.text.strip() for peak_position in peak_position_rows]

    # Get the list of weeks on chart
    weeks_on_chart_rows = soup.select('ul.o-chart-results-list-row > li:nth-child(4) > ul > li:nth-child(6) > span')
    weeks_on_chart = [weeks_on_chart.text.strip() for weeks_on_chart in weeks_on_chart_rows]

    # Create a dataframe with the extracted data
    billboard_df = pd.DataFrame({
        'this_week': this_week_positions,
        'artist': artists,
        'song': songs,
        'last_week': last_week_positions,
        'peak_position': peak_positions,
        'weeks_on_chart': weeks_on_chart
    })

    return billboard_df

In [22]:
billboard_df = get_billboard_chart_data('https://www.billboard.com/charts/hot-100/2023-03-04/')
print(billboard_df.head())

  this_week                                artist                 song  \
0         1                           Miley Cyrus              Flowers   
1         2                                   SZA            Kill Bill   
2         3            PinkPantheress & Ice Spice  Boy's A Liar, Pt. 2   
3         4  Metro Boomin, The Weeknd & 21 Savage             Creepin'   
4         5                         Morgan Wallen           Last Night   

  last_week peak_position weeks_on_chart  
0         1             1              6  
1         2             2             11  
2         4             3              3  
3         3             3             12  
4         5             3              4  


<h3>3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.</h3>

In [28]:
# Function that finds the next closest Saturday
def get_next_saturday(date):
    # If the date is a Saturday, return the date
    if date.weekday() == 5:
        return date
    # Otherwise, return the next Saturday
    else:
        return date + pd.Timedelta((5 - date.weekday()) % 7, unit='d')

In [52]:
# Last 10 weeks dataframe
past_10_weeks_billboard_df = pd.DataFrame()

saturday_date = get_next_saturday(pd.to_datetime('today'))
previous_date = saturday_date

# Loop through the past 10 weeks
for i in range(4):

    print('iteration: ' + str(i))
    print('Initial previous date: ' + str(previous_date))

    # Get and update the date
    if i == 0:
        date = saturday_date
    else:
        # Subtract a week from the latest date
        date = saturday_date - pd.DateOffset(weeks=i)

    print('Date: ' + str(date))
    print('Previous date: ' + str(previous_date))

    # Get the URL
    url = 'https://www.billboard.com/charts/hot-100/' + date.strftime('%Y-%m-%d')

    print('URL: ' + url)

    # Get the dataframe
    billboard_df = get_billboard_chart_data(url)

    # Append the dataframe to the past 10 weeks dataframe
    past_10_weeks_billboard_df = pd.concat([past_10_weeks_billboard_df, billboard_df])

iteration: 0
Initial previous date: 2023-05-13 19:40:52.292033
Date: 2023-05-13 19:40:52.292033
Previous date: 2023-05-13 19:40:52.292033
URL: https://www.billboard.com/charts/hot-100/2023-05-13
  this_week                       artist             song last_week  \
0         1                Morgan Wallen       Last Night         1   
1         2                          SZA        Kill Bill         2   
2         3                  Miley Cyrus          Flowers         3   
3         4  Eslabon Armado X Peso Pluma  Ella Baila Sola         4   
4         5          Rema & Selena Gomez        Calm Down         6   

  peak_position weeks_on_chart  
0             1             14  
1             1             21  
2             1             16  
3             4              7  
4             5             35  
iteration: 1
Initial previous date: 2023-05-13 19:40:52.292033
Date: 2023-05-06 19:40:52.292033
Previous date: 2023-05-13 19:40:52.292033
URL: https://www.billboard.com/charts/hot-